In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import numpy as np
from itertools import combinations, groupby
from collections import Counter

In [6]:
orders = pd.read_csv('all_orders.csv', index_col = 0)
cluster_df = pd.read_csv('cluster_df')
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
orders = orders.sort_values(['order_id','product_id'])
orders

,order_id,product_id,add_to_cart_order,reordered,aisle_id,department_id,aisle,department,user_id,cluster
30264547,1,10246,3,0,83,4,fresh vegetables,produce,112108,5
30264545,1,11109,2,1,108,16,other creams cheeses,dairy eggs,112108,5
30264548,1,13176,6,0,24,4,fresh fruits,produce,112108,5
30264544,1,22035,8,1,21,16,packaged cheese,dairy eggs,112108,5
30264550,1,43633,5,1,95,15,canned meat seafood,canned goods,112108,5
...,...,...,...,...,...,...,...,...,...,...
23108244,3421083,21162,3,0,92,18,baby food formula,babies,25247,5
23108238,3421083,24852,9,1,24,4,fresh fruits,produce,25247,5
23108247,3421083,35211,5,0,92,18,baby food formula,babies,25247,5
23108242,3421083,39678,6,1,74,17,dish detergents,household,25247,5


In [10]:
# Returns frequency counts for items and item pairs
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else: 
        return pd.Series(Counter(iterable)).rename("freq")

    
# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))


# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair
            

# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [11]:
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))


    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Filter from order_item items below min support 
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item) 


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)
    
    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])
    
    
    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [8]:
# separate order data by cluster and transform into expected format of group association function

cluster_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [12]:
# run each cluster series through association rule function and obtain association rules at item level

group_association_rules_dic = {}
for x in range(0,6):
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Starting order_item:                5714445
Items with support >= 0.0001:            7547
Remaining order_item:               5433087
Remaining orders with 2+ items:      518168
Remaining order_item:               5414155
Item pairs:                         5087275
Item pairs with support >= 0.0001:      91877

Starting order_item:                 219870
Items with support >= 0.0001:            4796
Remaining order_item:                204518
Remaining orders with 2+ items:       37565
Remaining order_item:                193295
Item pairs:                          220291
Item pairs with support >= 0.0001:      35858

Starting order_item:               10128193
Items with support >= 0.0001:           13288
Remaining order_item:               9182459
Remaining orders with 2+ items:      990115
Remaining order_item:               9106863
Item pairs:                        15985469
Item pairs with support >= 0.0001:      28487

Starting order_item:                 540699
Items with suppor

In [13]:
# format item association rule dataframes and merge product names

for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
               .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products[['product_id','product_name']]
               .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                    'product_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confidenceAtoB',
                                                                    'confidenceBtoA','lift']]
    

In [31]:
# example of association rule dataframe

x = group_association_rules_dic[5]
x.sort_values('lift', ascending = False)

,item_A,item_B,product_name_A,product_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,20153,46949,Eat Your Colors Purples Puree Baby Food,Eat Your Colors Reds Puree Baby Food,132,0.000103,283,0.000221,227,0.000177,0.466431,0.581498,2633.671331
1,29671,38652,Organic Bluephoria Yerba Mate,Yerba Mate Orange Exuberance Tea,160,0.000125,317,0.000247,297,0.000232,0.504732,0.538721,2178.232376
2,6583,38652,Oraganic Lemon Elation Yerba Mate Drink,Yerba Mate Orange Exuberance Tea,131,0.000102,294,0.000229,297,0.000232,0.445578,0.441077,1922.947617
5,6583,29671,Oraganic Lemon Elation Yerba Mate Drink,Organic Bluephoria Yerba Mate,130,0.000101,294,0.000229,317,0.000247,0.442177,0.410095,1787.873130
11,8833,9497,"Smoothie Fruits, Squished, The Green One, Over...",Smoothie Fruits Squished The Purple One Over 6...,196,0.000153,396,0.000309,366,0.000286,0.494949,0.535519,1733.323950
6,11187,29671,Revel Berry Yerba Mate,Organic Bluephoria Yerba Mate,176,0.000137,428,0.000334,317,0.000247,0.411215,0.555205,1662.683452
1925,41349,49570,Dairy Free Greek Yogurt Strawberry,Dairy Free Greek Yogurt Blueberry,191,0.000149,398,0.000311,378,0.000295,0.479899,0.505291,1627.265561
1927,43788,46352,Organic Pearfecto Level 1 Baby Food,Peachy Keen Organic Level 1,153,0.000119,374,0.000292,359,0.000280,0.409091,0.426184,1460.579894
1930,19097,47940,Light Harvest Peach Yogurt,Light Strawberry Yogurt,148,0.000115,347,0.000271,375,0.000293,0.426513,0.394667,1457.809952
1932,11224,39739,Organic Cashew Nondairy Blueberry Yogurt,Organic Nondairy Strawberry Cashew Yogurt,222,0.000173,359,0.000280,550,0.000429,0.618384,0.403636,1441.105495


In [15]:
# separate order data by cluster and transform into expected format of group association function at aisle level

cluster_aisle_orders = {}
for x in range(0,6):
    i = orders[orders['cluster'] == x]
    i = i.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')
    cluster_aisle_orders[x] = i

In [16]:
orders.groupby(['order_id','aisle_id'])['product_id'].count().reset_index().set_index('order_id')['aisle_id'].rename('item_id')

order_id
1           21
1           24
1           83
1           95
1          108
          ... 
3421083     52
3421083     74
3421083     78
3421083     92
3421083    117
Name: item_id, Length: 24329222, dtype: int64

In [17]:
orders[orders['cluster'] == 5]['order_id'].nunique()

1325895

In [18]:
# run association function at aisle level

group_aisle_association_rules_dic = {}
for x in range(0,6):
    group_aisle_association_rules_dic[x] = association_rules(cluster_aisle_orders[x],.0001)

Starting order_item:                3744330
Items with support >= 0.0001:             134
Remaining order_item:               3744330
Remaining orders with 2+ items:      518732
Remaining order_item:               3723763
Item pairs:                            8841
Item pairs with support >= 0.0001:       6251

Starting order_item:                 142874
Items with support >= 0.0001:             134
Remaining order_item:                142874
Remaining orders with 2+ items:       34461
Remaining order_item:                128100
Item pairs:                            6636
Item pairs with support >= 0.0001:       4178

Starting order_item:                7661793
Items with support >= 0.0001:             134
Remaining order_item:               7661793
Remaining orders with 2+ items:      989145
Remaining order_item:               7576779
Item pairs:                            8908
Item pairs with support >= 0.0001:       8209

Starting order_item:                 417170
Items with suppor

In [19]:
# reformat cluster aisle assocation dataframes

for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
               .rename(columns = {'aisle_id':'item_A','aisle':'aisle_name_A'}), on = 'item_A')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].merge(aisles[['aisle_id','aisle']]
               .rename(columns = {'aisle_id':'item_B','aisle':'aisle_name_B'}), on = 'item_B')
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x][['item_A','item_B','aisle_name_A',
                                                                    'aisle_name_B','freqAB','supportAB','freqA',
                                                                    'supportA','freqB','supportB','confidenceAtoB',
                                                                    'confidenceBtoA','lift']]
    group_aisle_association_rules_dic[x] = group_aisle_association_rules_dic[x].rename(columns = {'item_A':'aisle_A','item_B':'aisle_B'})

In [20]:
x = group_aisle_association_rules_dic[5].sort_values('lift',ascending = False)
x.sort_values('lift',ascending = False)

,aisle_A,aisle_B,aisle_name_A,aisle_name_B,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,28,62,red wines,white wines,1770,0.001376,7042,0.005476,6505,0.005059,0.251349,0.272098,49.687669
38,124,134,spirits,specialty wines champagnes,365,0.000284,5163,0.004015,2209,0.001718,0.070695,0.165233,41.154156
39,62,134,white wines,specialty wines champagnes,439,0.000341,6505,0.005059,2209,0.001718,0.067487,0.198732,39.286214
40,27,134,beers coolers,specialty wines champagnes,363,0.000282,5534,0.004303,2209,0.001718,0.065595,0.164328,38.184792
37,28,134,red wines,specialty wines champagnes,426,0.000331,7042,0.005476,2209,0.001718,0.060494,0.192847,35.215718
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,120,124,yogurt,spirits,686,0.000533,438578,0.341058,5163,0.004015,0.001564,0.132868,0.389577
146,91,124,soy lactosefree,spirits,413,0.000321,274969,0.213828,5163,0.004015,0.001502,0.079992,0.374096
5166,27,92,beers coolers,baby food formula,140,0.000109,5534,0.004303,90776,0.070591,0.025298,0.001542,0.358374
5165,28,92,red wines,baby food formula,159,0.000124,7042,0.005476,90776,0.070591,0.022579,0.001752,0.319852


In [24]:
cluster_df['cluster'].value_counts() / cluster_df['cluster'].value_counts().sum()

2    0.356114
5    0.353462
0    0.161244
4    0.072029
3    0.038519
1    0.018632
Name: cluster, dtype: float64

In [32]:
for x in group_association_rules_dic:
    group_association_rules_dic[x].to_csv('group'+str(x)+'_item_rules.csv')

In [33]:
for x in group_aisle_association_rules_dic:
    group_aisle_association_rules_dic[x].to_csv('group'+str(x)+'_aisle_rules.csv')